In [2]:
import pandas as pd
from sodapy import Socrata

MyAppToken='6v9nCt2L0Tftx31xuIhXQf5uv'
username='cz2527@columbia.edu'
password='Tools002'
client = Socrata('data.cityofnewyork.us',
                 MyAppToken,
                 username=username,
                 password=password)

results = client.get("8end-qv57",limit=3000)
df = pd.DataFrame.from_records(results)

df['event_name']= df['event_name'].apply(lambda x:x.lower())
df['event_id']= df['event_id'].apply(lambda x:int(x))

def change_type(results_df,keywords,event_type):
    for i in range(len(results_df)):
        if results_df.loc[i]['event_type']=='Special Event':
            for j in keywords:
                if j in results_df.loc[i]['event_name']:
                    results_df.loc[i]['event_type']=event_type
                    
change_type(df,('Closure','closure','CLOSED'),'Closure')
change_type(df,('Construction','construction'),'Construction')
change_type(df,('Maintenance',),'Maintenance')
change_type(df,('Farmers Market',),'Farmers Market')
#change_type(results_df,('Market',),'Special Market')
change_type(df,('Celebration','celebration'),'Celebration')


import datetime
import numpy as np
df['index']=df.index
df['start_date_time']=df['start_date_time'].apply(lambda x: np.datetime64(x))
df['end_date_time']=df['end_date_time'].apply(lambda x: np.datetime64(x))
df['weekday']=df['start_date_time'].apply(lambda x: int(x.weekday())+1)

df_show=df.iloc[:,[-2,6,3,5,-6,-4,1,-1]]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
df=df_show

In [4]:
df['weekday']=df['weekday'].astype('str')
df['start_date_time']=df['start_date_time'].apply(lambda x:x.ctime())
df['end_date_time']=df['end_date_time'].apply(lambda x:x.ctime())

In [7]:
from bokeh.layouts import row, widgetbox,layout
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.widgets import RangeSlider, Button, DataTable, TableColumn, NumberFormatter,Select,TextInput
from bokeh.io import curdoc
doc=curdoc()
def modify_doc(doc):
    source = ColumnDataSource(data=dict())



    type_ = Select(title="Event Type", value="All",options=list(df['event_type'].unique()))
    name = TextInput(title="Events name contains")
    borough = Select(title="Event Borough", value="All",options=list(df['event_borough'].unique()))
    weekday = Select(title="Event Day", value="All",options=list(df['weekday'].unique()))

    controls = [type_, name, borough, weekday]
    for control in controls:
        control.on_change('value', lambda attr, old, new: update())


    def select_events():
        type_val = type_.value
        borough_val = borough.value
        weekday_val = weekday.value
        name_val = name.value.strip()
        selected = df
        if (type_val != "All"):
            selected = selected[selected.event_type.str.contains(type_val)==True]
        if (borough_val != "All"):
            selected = selected[selected.event_borough.str.contains(borough_val)==True]
        if (weekday_val != "All"):
            selected = selected[selected.weekday.str.contains(weekday_val)==True]
        if (name_val != ""):
            selected = selected[selected.event_name.str.contains(name_val)==True]
        return selected

    def update():
        df=select_events()
        source.data = {
            'Event Name'             : df.event_name,
            'Borough'           : df.event_borough,
            'Location' : df.event_location,
            'Type' : df.event_type,
            'Start Time':df.start_date_time,
            'End Time':df.end_date_time,
            'Weekday':df.weekday
        }



    columns = [
        TableColumn(field="Event Name", title="Event Name"),
        TableColumn(field="Borough", title="Borough"),
        TableColumn(field="Location", title="Location"),
        TableColumn(field="Type", title="Type"),
        TableColumn(field="Start Time", title="Start Time"),
        TableColumn(field="End Time", title="End Time"),
        TableColumn(field="Weekday", title="Weekday")
    ]

    data_table = DataTable(source=source, columns=columns, width=800)


    controls_ = widgetbox(type_, name, borough, weekday)
    table = widgetbox(data_table)

    curdoc().add_root(row(controls_, table))
    curdoc().title = "Select Events"

    update()

SyntaxError: invalid syntax (<ipython-input-7-7fd8b9223868>, line 6)

In [6]:
from bokeh.io import output_notebook, show
output_notebook()
show(curdoc)

Loading BokehJS ...

ValueError: Callback functions must have signature func(doc), got func()